In [1]:
import pandas as pd
import zipfile
from io import BytesIO
import requests

In [2]:
varlist = set(['Población de 0 - 14 años', 'Población de 15 - 64 años', 'Población de 65 años y más', 'Población total', 
               'Provincia / Cantón', 'Total de Escuelas', 'Total de Kinder', 'Total de colegios', 'Año'])

In [3]:
demographics_df_list = []
column_list = set()
for yr in [str(i).zfill(2) for i in range(6, 21)]: 
    for province in ['san_jose', 'alajuela', 'cartago', 'heredia', 'guanacaste', 'puntarenas', 'limon']:
        # Downloading the file by sending the request to the URL
        req = requests.get(f"https://www.ccss.sa.cr/arc/estadisticas/actuarial/67/{province}_{yr}.zip")

        with zipfile.ZipFile(BytesIO(req.content)) as z:
            # open the csv file in the dataset
            with z.open(z.namelist()[0]) as f: 
                # read the dataset
                df = pd.read_excel(f.read(), skiprows=1)
                df["Año"] = int("20" + yr)
                df.columns = df.columns.str.replace(r"/[0-9]", "",  regex=True)
                df.columns = df.columns.str.replace(r"\s\s+", " ",  regex=True)
                df.columns = df.columns.str.strip()
                var_diff = set(df.columns) - varlist
                df.drop(columns=var_diff, inplace=True)
                demographics_df_list.append(df)

In [4]:
demographics_df = pd.concat(demographics_df_list, ignore_index=True)

In [5]:
select_municipalities = demographics_df["Provincia / Cantón"].str.contains(r"^\d{3}:", regex=True, na=False)

In [6]:
demographics_df = demographics_df[select_municipalities].copy()

In [7]:
demographics_df.rename(columns={'Provincia / Cantón': 'municipality', 
                                'Población de 0 - 14 años': "pop_014", 
                                'Población de 15 - 64 años': "pop_1564", 
                                'Población de 65 años y más': "pop_65plus", 
                                'Población total': "pop", 
                                'Total de Kinder': "n_preschools", 
                                'Total de Escuelas': "n_schools", 
                                'Total de colegios': "n_highschools", 
                                'Año': "year"}, 
                        inplace=True)

In [8]:
demographics_df["pop_share014"] = demographics_df["pop_014"] / demographics_df["pop"]
demographics_df["pop_share65plus"] = demographics_df["pop_65plus"] / demographics_df["pop"]
demographics_df["k_12centers"] = demographics_df["n_preschools"] + demographics_df["n_schools"] + demographics_df["n_highschools"]

In [9]:
demographics_df = demographics_df[["municipality", "year", "pop_share014", "pop_share65plus", "k_12centers", "pop"]].copy()

In [10]:
demographics_df.to_stata("population.dta", write_index=False)